<a href="https://colab.research.google.com/github/Kridr/Momentum-Strategy-Crypto/blob/main/momentum_strategy_crypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing neccesarry libraries

Installing pycoingecko. We can use API via requests library and RapidAPI, but it is much more convenient to use library. Primarily this notebook is about strategy

In [ ]:
!pip install pycoingecko

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


We will need some libraries for data analisys

In [ ]:
import pandas as pd
import numpy as np

from pycoingecko import CoinGeckoAPI
import time
from scipy.stats import percentileofscore

# Collecting data

Initializing CoinGecko API

In [ ]:
cg = CoinGeckoAPI()

Collecting market info about all coins, then filter them by current price in USD. We want current price to be NOT less than 5 USD (I assume other coins are just trash)

In [ ]:
coins_markets = cg.get_coins_markets(vs_currency='usd')

coins_markets = list(filter(lambda x: x['current_price'] >= 10, coins_markets))
id_to_market = {x['id']: x for x in coins_markets}

Get dates for obtaining historical data. Because of datatypes I cannot directly subtract `current_date` and `timedelta['M']`, so I lost some accuracy in dates. But I think it does not really matter

In [ ]:
current_date = np.datetime64('today')

cdate_12m = (current_date - 12 * np.timedelta64(30, 'D')).item().strftime('%d-%m-%Y')
cdate_6m = (current_date - 6 * np.timedelta64(30, 'D')).item().strftime('%d-%m-%Y')
cdate_3m = (current_date - 3 * np.timedelta64(30, 'D')).item().strftime('%d-%m-%Y')
cdate_1m = (current_date - np.timedelta64(30, 'D')).item().strftime('%d-%m-%Y')

print(current_date)
print(cdate_12m, cdate_6m, cdate_3m, cdate_1m)

2022-08-27
01-09-2021 28-02-2022 29-05-2022 28-07-2022


Now we got all the information, so we need to create df first

In [ ]:
columns = [
    'coin',
    'market_cap',
    'current_price',
    '12m_before_price',
    '12m_ratio',
    '12m_score',
    '6m_before_price',
    '6m_ratio',
    '6m_score',
    '3m_before_price',
    '3m_ratio',
    '3m_score',
    '1m_before_price',
    '1m_ratio',
    '1m_score'
]
df = pd.concat(
    [
        pd.Series(
            [
                market_data['id'],
                market_data['market_cap'],
                market_data['current_price'],
            ] + [np.nan] * 12
        )
        for market_data in id_to_market.values()
    ], 
    ignore_index=True,
    axis=1
).T
df.set_axis(columns, axis=1, inplace=True)
df.set_index('coin', inplace=True)
df.head()

,market_cap,current_price,12m_before_price,12m_ratio,12m_score,6m_before_price,6m_ratio,6m_score,3m_before_price,3m_ratio,3m_score,1m_before_price,1m_ratio,1m_score
coin,,,,,,,,,,,,,,
bitcoin,387092297634,20231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ethereum,181223174531,1506.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
binancecoin,46248386522,283.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
solana,11203731642,32.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
staked-ether,6283975408,1464.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Filling df with historical data. As long as we use free API, we cannot make more than 50 request calls per minute. So we need to use `sleep()` method

In [ ]:
colname_to_date = {
    '12m_before_price': cdate_12m,
    '6m_before_price': cdate_6m,
    '3m_before_price': cdate_3m,
    '1m_before_price': cdate_1m
}

for id in df.index:
    print(f'{id} is processing...')
    for col, date in colname_to_date.items():
        history_data = cg.get_coin_history_by_id(id=id, 
                                                 date=date,
                                                 localization='false'
                                                )
        try:
            df.at[id, col] = history_data['market_data']['current_price']['usd']
        except:
            df.at[id, col] = np.nan
    time.sleep(15)
df

bitcoin is processing...
ethereum is processing...
binancecoin is processing...
solana is processing...
staked-ether is processing...
avalanche-2 is processing...
wrapped-bitcoin is processing...
ethereum-classic is processing...
okb is processing...
litecoin is processing...
ftx-token is processing...
cosmos is processing...
monero is processing...
bitcoin-cash is processing...
quant-network is processing...
axie-infinity is processing...
elrond-erd-2 is processing...
aave is processing...
bitcoin-cash-sv is processing...
zcash is processing...
huobi-btc is processing...
compound-ether is processing...
maker is processing...
pax-gold is processing...
arweave is processing...
dash is processing...
tether-gold is processing...
bitcoin-gold is processing...


,market_cap,current_price,12m_before_price,12m_ratio,12m_score,6m_before_price,6m_ratio,6m_score,3m_before_price,3m_ratio,3m_score,1m_before_price,1m_ratio,1m_score
coin,,,,,,,,,,,,,,
bitcoin,387092297634,20231,47335.420299,NaN,NaN,37803.59016,NaN,NaN,29088.236453,NaN,NaN,22908.88426,NaN,NaN
ethereum,181223174531,1506.55,3440.562336,NaN,NaN,2629.483121,NaN,NaN,1798.694855,NaN,NaN,1636.949181,NaN,NaN
binancecoin,46248386522,283.25,466.325068,NaN,NaN,361.294866,NaN,NaN,309.03661,NaN,NaN,270.305244,NaN,NaN
solana,11203731642,32.07,108.706376,NaN,NaN,85.568197,NaN,NaN,44.464812,NaN,NaN,40.295449,NaN,NaN
staked-ether,6283975408,1464.2,3419.056919,NaN,NaN,2625.816059,NaN,NaN,1757.33212,NaN,NaN,1602.692468,NaN,NaN
avalanche-2,5886542012,20.65,39.622302,NaN,NaN,74.943551,NaN,NaN,24.888556,NaN,NaN,23.437344,NaN,NaN
wrapped-bitcoin,5008847839,20225,47389.375376,NaN,NaN,37783.293367,NaN,NaN,29086.382021,NaN,NaN,22946.67977,NaN,NaN
ethereum-classic,4609164860,33.74,64.092154,NaN,NaN,27.4961,NaN,NaN,22.823249,NaN,NaN,32.803053,NaN,NaN
okb,4143747494,16.15,20.465788,NaN,NaN,16.862746,NaN,NaN,11.532209,NaN,NaN,15.922691,NaN,NaN


In [ ]:
df.to_csv('df.csv')

In [ ]:
df = pd.read_csv('df.csv')
df.set_index('coin', inplace=True)

# Calculating ratios and scores for each historical point 

Dropping coins if some of historical data is missing

In [ ]:
df.dropna(subset=colname_to_date.keys(), inplace=True)

Calculating ratios

In [ ]:
ratio_to_price = {
    '12m_ratio': '12m_before_price',
    '6m_ratio': '6m_before_price',
    '3m_ratio': '3m_before_price',
    '1m_ratio': '1m_before_price'
}

for id in df.index:
    for ratio_col, hprice_col in ratio_to_price.items():
        df.at[id, ratio_col] = df.at[id, 'current_price'] / df.at[id, hprice_col]

df

,market_cap,current_price,12m_before_price,12m_ratio,12m_score,6m_before_price,6m_ratio,6m_score,3m_before_price,3m_ratio,3m_score,1m_before_price,1m_ratio,1m_score
coin,,,,,,,,,,,,,,
bitcoin,387092297634,20231.00,47335.420299,0.427397,NaN,37803.590160,0.535161,NaN,29088.236453,0.695505,NaN,22908.884260,0.883107,NaN
ethereum,181223174531,1506.55,3440.562336,0.437879,NaN,2629.483121,0.572945,NaN,1798.694855,0.837580,NaN,1636.949181,0.920340,NaN
binancecoin,46248386522,283.25,466.325068,0.607409,NaN,361.294866,0.783986,NaN,309.036610,0.916558,NaN,270.305244,1.047889,NaN
solana,11203731642,32.07,108.706376,0.295015,NaN,85.568197,0.374789,NaN,44.464812,0.721244,NaN,40.295449,0.795872,NaN
staked-ether,6283975408,1464.20,3419.056919,0.428247,NaN,2625.816059,0.557617,NaN,1757.332120,0.833195,NaN,1602.692468,0.913588,NaN
avalanche-2,5886542012,20.65,39.622302,0.521171,NaN,74.943551,0.275541,NaN,24.888556,0.829699,NaN,23.437344,0.881073,NaN
wrapped-bitcoin,5008847839,20225.00,47389.375376,0.426783,NaN,37783.293367,0.535289,NaN,29086.382021,0.695343,NaN,22946.679770,0.881391,NaN
ethereum-classic,4609164860,33.74,64.092154,0.526429,NaN,27.496100,1.227083,NaN,22.823249,1.478317,NaN,32.803053,1.028563,NaN
okb,4143747494,16.15,20.465788,0.789122,NaN,16.862746,0.957733,NaN,11.532209,1.400426,NaN,15.922691,1.014276,NaN


Calculating scores

In [ ]:
score_to_ratio = {
    '12m_score': '12m_ratio',
    '6m_score': '6m_ratio',
    '3m_score': '3m_ratio',
    '1m_score': '1m_ratio'
}

for id in df.index:
    for score_col, ratio_col in score_to_ratio.items():
        df.at[id, score_col] = percentileofscore(df[ratio_col], df.at[id, ratio_col]) / 100
df

,market_cap,current_price,12m_before_price,12m_ratio,12m_score,6m_before_price,6m_ratio,6m_score,3m_before_price,3m_ratio,3m_score,1m_before_price,1m_ratio,1m_score
coin,,,,,,,,,,,,,,
bitcoin,387092297634,20231.00,47335.420299,0.427397,0.535714,37803.590160,0.535161,0.428571,29088.236453,0.695505,0.178571,22908.884260,0.883107,0.321429
ethereum,181223174531,1506.55,3440.562336,0.437879,0.642857,2629.483121,0.572945,0.535714,1798.694855,0.837580,0.535714,1636.949181,0.920340,0.571429
binancecoin,46248386522,283.25,466.325068,0.607409,0.892857,361.294866,0.783986,0.750000,309.036610,0.916558,0.678571,270.305244,1.047889,0.964286
solana,11203731642,32.07,108.706376,0.295015,0.250000,85.568197,0.374789,0.107143,44.464812,0.721244,0.250000,40.295449,0.795872,0.107143
staked-ether,6283975408,1464.20,3419.056919,0.428247,0.571429,2625.816059,0.557617,0.500000,1757.332120,0.833195,0.464286,1602.692468,0.913588,0.500000
avalanche-2,5886542012,20.65,39.622302,0.521171,0.750000,74.943551,0.275541,0.035714,24.888556,0.829699,0.428571,23.437344,0.881073,0.250000
wrapped-bitcoin,5008847839,20225.00,47389.375376,0.426783,0.500000,37783.293367,0.535289,0.464286,29086.382021,0.695343,0.142857,22946.679770,0.881391,0.285714
ethereum-classic,4609164860,33.74,64.092154,0.526429,0.785714,27.496100,1.227083,1.000000,22.823249,1.478317,0.964286,32.803053,1.028563,0.892857
okb,4143747494,16.15,20.465788,0.789122,0.928571,16.862746,0.957733,0.964286,11.532209,1.400426,0.928571,15.922691,1.014276,0.857143


Calculate total ratio_score

In [ ]:
df['ratio_score'] = 0.1 * df['12m_score'] + \
                    0.2 * df['6m_score'] + \
                    0.3 * df['3m_score'] + \
                    0.4 * df['1m_score']
df['ratio_score'] = df['ratio_score'] / df['ratio_score'].sum()
df

,market_cap,current_price,12m_before_price,12m_ratio,12m_score,6m_before_price,6m_ratio,6m_score,3m_before_price,3m_ratio,3m_score,1m_before_price,1m_ratio,1m_score,ratio_score
coin,,,,,,,,,,,,,,,
bitcoin,387092297634,20231.00,47335.420299,0.427397,0.535714,37803.590160,0.535161,0.428571,29088.236453,0.695505,0.178571,22908.884260,0.883107,0.321429,0.022167
ethereum,181223174531,1506.55,3440.562336,0.437879,0.642857,2629.483121,0.572945,0.535714,1798.694855,0.837580,0.535714,1636.949181,0.920340,0.571429,0.038670
binancecoin,46248386522,283.25,466.325068,0.607409,0.892857,361.294866,0.783986,0.750000,309.036610,0.916558,0.678571,270.305244,1.047889,0.964286,0.057143
solana,11203731642,32.07,108.706376,0.295015,0.250000,85.568197,0.374789,0.107143,44.464812,0.721244,0.250000,40.295449,0.795872,0.107143,0.011330
staked-ether,6283975408,1464.20,3419.056919,0.428247,0.571429,2625.816059,0.557617,0.500000,1757.332120,0.833195,0.464286,1602.692468,0.913588,0.500000,0.034236
avalanche-2,5886542012,20.65,39.622302,0.521171,0.750000,74.943551,0.275541,0.035714,24.888556,0.829699,0.428571,23.437344,0.881073,0.250000,0.021429
wrapped-bitcoin,5008847839,20225.00,47389.375376,0.426783,0.500000,37783.293367,0.535289,0.464286,29086.382021,0.695343,0.142857,22946.679770,0.881391,0.285714,0.020690
ethereum-classic,4609164860,33.74,64.092154,0.526429,0.785714,27.496100,1.227083,1.000000,22.823249,1.478317,0.964286,32.803053,1.028563,0.892857,0.063793
okb,4143747494,16.15,20.465788,0.789122,0.928571,16.862746,0.957733,0.964286,11.532209,1.400426,0.928571,15.922691,1.014276,0.857143,0.062562


Calculating total market score

In [ ]:
df['market_score'] = df['market_cap'] / df['market_cap'].sum()
df

,market_cap,current_price,12m_before_price,12m_ratio,12m_score,6m_before_price,6m_ratio,6m_score,3m_before_price,3m_ratio,3m_score,1m_before_price,1m_ratio,1m_score,ratio_score,market_score
coin,,,,,,,,,,,,,,,,
bitcoin,387092297634,20231.00,47335.420299,0.427397,0.535714,37803.590160,0.535161,0.428571,29088.236453,0.695505,0.178571,22908.884260,0.883107,0.321429,0.022167,0.570556
ethereum,181223174531,1506.55,3440.562336,0.437879,0.642857,2629.483121,0.572945,0.535714,1798.694855,0.837580,0.535714,1636.949181,0.920340,0.571429,0.038670,0.267115
binancecoin,46248386522,283.25,466.325068,0.607409,0.892857,361.294866,0.783986,0.750000,309.036610,0.916558,0.678571,270.305244,1.047889,0.964286,0.057143,0.068168
solana,11203731642,32.07,108.706376,0.295015,0.250000,85.568197,0.374789,0.107143,44.464812,0.721244,0.250000,40.295449,0.795872,0.107143,0.011330,0.016514
staked-ether,6283975408,1464.20,3419.056919,0.428247,0.571429,2625.816059,0.557617,0.500000,1757.332120,0.833195,0.464286,1602.692468,0.913588,0.500000,0.034236,0.009262
avalanche-2,5886542012,20.65,39.622302,0.521171,0.750000,74.943551,0.275541,0.035714,24.888556,0.829699,0.428571,23.437344,0.881073,0.250000,0.021429,0.008676
wrapped-bitcoin,5008847839,20225.00,47389.375376,0.426783,0.500000,37783.293367,0.535289,0.464286,29086.382021,0.695343,0.142857,22946.679770,0.881391,0.285714,0.020690,0.007383
ethereum-classic,4609164860,33.74,64.092154,0.526429,0.785714,27.496100,1.227083,1.000000,22.823249,1.478317,0.964286,32.803053,1.028563,0.892857,0.063793,0.006794
okb,4143747494,16.15,20.465788,0.789122,0.928571,16.862746,0.957733,0.964286,11.532209,1.400426,0.928571,15.922691,1.014276,0.857143,0.062562,0.006108


Calculating total score based on market score and ratio score

In [ ]:
df['total_score'] = df['ratio_score'] * 0.95 + df['market_score'] * 0.05
df

,market_cap,current_price,12m_before_price,12m_ratio,12m_score,6m_before_price,6m_ratio,6m_score,3m_before_price,3m_ratio,3m_score,1m_before_price,1m_ratio,1m_score,ratio_score,market_score,total_score
coin,,,,,,,,,,,,,,,,,
bitcoin,387092297634,20231.00,47335.420299,0.427397,0.535714,37803.590160,0.535161,0.428571,29088.236453,0.695505,0.178571,22908.884260,0.883107,0.321429,0.022167,0.570556,0.049587
ethereum,181223174531,1506.55,3440.562336,0.437879,0.642857,2629.483121,0.572945,0.535714,1798.694855,0.837580,0.535714,1636.949181,0.920340,0.571429,0.038670,0.267115,0.050092
binancecoin,46248386522,283.25,466.325068,0.607409,0.892857,361.294866,0.783986,0.750000,309.036610,0.916558,0.678571,270.305244,1.047889,0.964286,0.057143,0.068168,0.057694
solana,11203731642,32.07,108.706376,0.295015,0.250000,85.568197,0.374789,0.107143,44.464812,0.721244,0.250000,40.295449,0.795872,0.107143,0.011330,0.016514,0.011589
staked-ether,6283975408,1464.20,3419.056919,0.428247,0.571429,2625.816059,0.557617,0.500000,1757.332120,0.833195,0.464286,1602.692468,0.913588,0.500000,0.034236,0.009262,0.032988
avalanche-2,5886542012,20.65,39.622302,0.521171,0.750000,74.943551,0.275541,0.035714,24.888556,0.829699,0.428571,23.437344,0.881073,0.250000,0.021429,0.008676,0.020791
wrapped-bitcoin,5008847839,20225.00,47389.375376,0.426783,0.500000,37783.293367,0.535289,0.464286,29086.382021,0.695343,0.142857,22946.679770,0.881391,0.285714,0.020690,0.007383,0.020024
ethereum-classic,4609164860,33.74,64.092154,0.526429,0.785714,27.496100,1.227083,1.000000,22.823249,1.478317,0.964286,32.803053,1.028563,0.892857,0.063793,0.006794,0.060943
okb,4143747494,16.15,20.465788,0.789122,0.928571,16.862746,0.957733,0.964286,11.532209,1.400426,0.928571,15.922691,1.014276,0.857143,0.062562,0.006108,0.059739


We want to have top 15 coins (by total_score)

# Choosing 15 best coins and calculating total weights and number of coins to buy

In [ ]:
df = df.sort_values('total_score', ascending=False)[:15]
df

,market_cap,current_price,12m_before_price,12m_ratio,12m_score,6m_before_price,6m_ratio,6m_score,3m_before_price,3m_ratio,3m_score,1m_before_price,1m_ratio,1m_score,ratio_score,market_score,total_score
coin,,,,,,,,,,,,,,,,,
quant-network,1341398066,99.98,182.232542,0.548640,0.857143,105.633331,0.946482,0.892857,65.538194,1.525523,1.000000,96.832959,1.032500,0.928571,0.064532,0.001977,0.061404
ethereum-classic,4609164860,33.74,64.092154,0.526429,0.785714,27.496100,1.227083,1.000000,22.823249,1.478317,0.964286,32.803053,1.028563,0.892857,0.063793,0.006794,0.060943
okb,4143747494,16.15,20.465788,0.789122,0.928571,16.862746,0.957733,0.964286,11.532209,1.400426,0.928571,15.922691,1.014276,0.857143,0.062562,0.006108,0.059739
binancecoin,46248386522,283.25,466.325068,0.607409,0.892857,361.294866,0.783986,0.750000,309.036610,0.916558,0.678571,270.305244,1.047889,0.964286,0.057143,0.068168,0.057694
pax-gold,588595110,1730.71,1821.136620,0.950346,1.000000,1912.015806,0.905176,0.857143,1857.568032,0.931707,0.750000,1739.614673,0.994881,0.750000,0.054926,0.000868,0.052223
tether-gold,422248202,1712.81,1814.058486,0.944187,0.964286,1916.683105,0.893632,0.821429,1865.453180,0.918174,0.714286,1710.971939,1.001074,0.785714,0.054433,0.000622,0.051743
cosmos,3185370245,10.88,23.006582,0.472908,0.678571,26.803175,0.405922,0.250000,9.442714,1.152211,0.892857,10.322293,1.054029,1.000000,0.054187,0.004695,0.051713
ethereum,181223174531,1506.55,3440.562336,0.437879,0.642857,2629.483121,0.572945,0.535714,1798.694855,0.837580,0.535714,1636.949181,0.920340,0.571429,0.038670,0.267115,0.050092
bitcoin,387092297634,20231.00,47335.420299,0.427397,0.535714,37803.590160,0.535161,0.428571,29088.236453,0.695505,0.178571,22908.884260,0.883107,0.321429,0.022167,0.570556,0.049587


We need to recalculate scores to obtrain correct weights

In [ ]:
for id in df.index:
    for score_col, ratio_col in score_to_ratio.items():
        df.at[id, score_col] = percentileofscore(df[ratio_col], df.at[id, ratio_col]) / 100

df['ratio_score'] = 0.1 * df['12m_score'] + \
                    0.2 * df['6m_score'] + \
                    0.3 * df['3m_score'] + \
                    0.4 * df['1m_score']
df['ratio_score'] = df['ratio_score'] / df['ratio_score'].sum()

df['market_score'] = df['market_cap'] / df['market_cap'].sum()

df['total_score'] = df['ratio_score'] * 0.9 + df['market_score'] * 0.1

Introducing portfolio size

In [ ]:
portfolio_size = 1_000_000 #USD

Calculate number of coins to buy

In [ ]:
df['coins_to_buy'] = df['total_score'] * portfolio_size / df['current_price']
df

,market_cap,current_price,12m_before_price,12m_ratio,12m_score,6m_before_price,6m_ratio,6m_score,3m_before_price,3m_ratio,3m_score,1m_before_price,1m_ratio,1m_score,ratio_score,market_score,total_score,coins_to_buy
coin,,,,,,,,,,,,,,,,,,
quant-network,1341398066,99.98,182.232542,0.548640,0.733333,105.633331,0.946482,0.800000,65.538194,1.525523,1.000000,96.832959,1.032500,0.866667,0.110000,0.002103,0.099210,992.301202
ethereum-classic,4609164860,33.74,64.092154,0.526429,0.600000,27.496100,1.227083,1.000000,22.823249,1.478317,0.933333,32.803053,1.028563,0.800000,0.107500,0.007225,0.097473,2888.930675
okb,4143747494,16.15,20.465788,0.789122,0.866667,16.862746,0.957733,0.933333,11.532209,1.400426,0.866667,15.922691,1.014276,0.733333,0.103333,0.006496,0.093650,5798.734571
binancecoin,46248386522,283.25,466.325068,0.607409,0.800000,361.294866,0.783986,0.533333,309.036610,0.916558,0.400000,270.305244,1.047889,0.933333,0.085000,0.072498,0.083750,295.674420
pax-gold,588595110,1730.71,1821.136620,0.950346,1.000000,1912.015806,0.905176,0.733333,1857.568032,0.931707,0.533333,1739.614673,0.994881,0.533333,0.077500,0.000923,0.069842,40.354691
tether-gold,422248202,1712.81,1814.058486,0.944187,0.933333,1916.683105,0.893632,0.666667,1865.453180,0.918174,0.466667,1710.971939,1.001074,0.600000,0.075833,0.000662,0.068316,39.885446
cosmos,3185370245,10.88,23.006582,0.472908,0.466667,26.803175,0.405922,0.066667,9.442714,1.152211,0.800000,10.322293,1.054029,1.000000,0.087500,0.004993,0.079249,7283.945822
ethereum,181223174531,1506.55,3440.562336,0.437879,0.400000,2629.483121,0.572945,0.200000,1798.694855,0.837580,0.266667,1636.949181,0.920340,0.333333,0.036667,0.284081,0.061408,40.760734
bitcoin,387092297634,20231.00,47335.420299,0.427397,0.266667,37803.590160,0.535161,0.133333,29088.236453,0.695505,0.066667,22908.884260,0.883107,0.066667,0.012500,0.606796,0.071930,3.555415
